In [1]:
import sys
print(sys.executable)

/projectnb/selneuro/students/bsd112/.conda/envs/selneuro_base_env/bin/python


In [9]:
#pip install -e /projectnb/selneuro/Shankar/code/Python/map_my_cells/cell_type_mapper

In [4]:
#!pip install -U git+https://github.com/alleninstitute/abc_atlas_access >& /projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/junk.txt


In [5]:
#get_ipython().kernel.do_shutdown(restart=True)


{'status': 'ok', 'restart': True}

In [3]:
pwd

'/projectnb/selneuro/Shankar/code/Python/map_my_cells'

In [2]:
import anndata
import copy
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pathlib
import tempfile

from abc_atlas_access.abc_atlas_cache.abc_project_cache import AbcProjectCache

In [4]:
os.environ['NUMEXPR_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'

In [5]:
abc_data_dir = pathlib.Path('/projectnb/selneuro/Shankar/code/Python/map_my_cells/data/abc_atlas_data')
scratch_dir = pathlib.Path('/projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch')
precompute_dir = scratch_dir / 'precompute'
reference_dir = scratch_dir / 'reference'
query_dir = scratch_dir / 'query'

for dir_name in (precompute_dir, reference_dir, query_dir):
    if not dir_name.exists():
        dir_name.mkdir()

print(f'abc_data_dir: {abc_data_dir.absolute()}')
print(f'scratch_dir: {scratch_dir.absolute()}')

abc_data_dir: /projectnb/selneuro/Shankar/code/Python/map_my_cells/data/abc_atlas_data
scratch_dir: /projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch


In [9]:
abc_cache = AbcProjectCache.from_cache_dir(abc_data_dir)
abc_cache.load_latest_manifest()


In [10]:
abc_cache.list_metadata_files(directory='WMB-10X')


['cell_metadata',
 'cell_metadata_with_cluster_annotation',
 'example_genes_all_cells_expression',
 'gene',
 'region_of_interest_metadata']

In [11]:
%%time
abc_cache.get_directory_metadata(directory='WMB-taxonomy')
abc_cache.get_metadata_path(directory='WMB-10X', file_name='cell_metadata')

CPU times: user 5.53 ms, sys: 1.14 ms, total: 6.68 ms
Wall time: 6.81 ms


PosixPath('/projectnb/selneuro/Shankar/code/Python/map_my_cells/data/abc_atlas_data/metadata/WMB-10X/20241115/cell_metadata.csv')

In [12]:
cell_metadata = abc_cache.get_metadata_dataframe(
    directory='WMB-10X',
    file_name='cell_metadata'
)

In [14]:
for label in cell_metadata.feature_matrix_label.unique():
    print(label)

WMB-10Xv3-HPF
WMB-10Xv3-Isocortex-1
WMB-10Xv3-PAL
WMB-10Xv3-STR
WMB-10Xv3-CTXsp
WMB-10Xv3-HY
WMB-10Xv3-OLF
WMB-10Xv3-TH
WMB-10Xv3-P
WMB-10Xv3-MB
WMB-10Xv3-MY
WMB-10Xv3-CB
WMB-10Xv3-Isocortex-2
WMB-10Xv2-HPF
WMB-10Xv2-Isocortex-1
WMB-10Xv2-HY
WMB-10Xv2-TH
WMB-10Xv2-OLF
WMB-10Xv2-CTXsp
WMB-10Xv2-MB
WMB-10Xv2-Isocortex-2
WMB-10Xv2-Isocortex-3
WMB-10Xv2-Isocortex-4
WMB-10XMulti


In [15]:
isocortex_cells = cell_metadata[
    cell_metadata.feature_matrix_label.str.contains('Isocortex') |
    cell_metadata.feature_matrix_label.str.contains('HPF') |
    cell_metadata.feature_matrix_label.str.contains('OLF')
]

In [19]:
#isocortex_cells

In [20]:
def directory_from_feature_matrix_label(feature_matrix_label):
    """
    A utility function to take a feature_matrix_label and return the 'directory'
    in the abc_atlas_access cache where that feature matrix is stored
    """
    params = feature_matrix_label.split('-')
    return f'{params[0]}-{params[1]}'

In [21]:
%%time
h5ad_path_list = []
for feature_matrix in set(isocortex_cells.feature_matrix_label.values):
    directory = directory_from_feature_matrix_label(feature_matrix)
    h5ad_path = abc_cache.get_data_path(directory=directory, file_name=f'{feature_matrix}/raw')
    h5ad_path_list.append(str(h5ad_path))

WMB-10Xv3-Isocortex-1-raw.h5ad: 100%|██████████| 11.8G/11.8G [08:30<00:00, 23.0MMB/s]    
WMB-10Xv3-OLF-raw.h5ad: 100%|██████████| 3.11G/3.11G [02:26<00:00, 21.2MMB/s]    
WMB-10Xv3-HPF-raw.h5ad: 100%|██████████| 7.41G/7.41G [05:32<00:00, 22.3MMB/s]    
WMB-10Xv2-Isocortex-1-raw.h5ad: 100%|██████████| 8.60G/8.60G [06:31<00:00, 22.0MMB/s]    
WMB-10Xv2-OLF-raw.h5ad: 100%|██████████| 5.13G/5.13G [03:29<00:00, 24.4MMB/s]    
WMB-10Xv2-Isocortex-3-raw.h5ad: 100%|██████████| 8.46G/8.46G [06:14<00:00, 22.6MMB/s]    
WMB-10Xv2-Isocortex-2-raw.h5ad: 100%|██████████| 9.44G/9.44G [07:45<00:00, 20.3MMB/s]    
WMB-10Xv2-Isocortex-4-raw.h5ad: 100%|██████████| 8.69G/8.69G [12:38<00:00, 11.5MMB/s]    
WMB-10Xv3-Isocortex-2-raw.h5ad: 100%|██████████| 8.36G/8.36G [06:38<00:00, 21.0MMB/s]    
WMB-10Xv2-HPF-raw.h5ad: 100%|██████████| 6.10G/6.10G [04:17<00:00, 23.7MMB/s]    

CPU times: user 26min 11s, sys: 2min 59s, total: 29min 10s
Wall time: 1h 4min 9s


In [24]:
training_set_path = scratch_dir / 'training_cell_metadata.csv'
isocortex_cells.to_csv(training_set_path, index=False)

In [22]:
from cell_type_mapper.cli.precompute_stats_abc import (
    PrecomputationABCRunner
)


cluster_annotation_term.csv: 100%|██████████| 861k/861k [00:00<00:00, 1.49MMB/s] 
cluster_to_cluster_annotation_membership.csv: 100%|██████████| 2.21M/2.21M [00:00<00:00, 7.17MMB/s]
UsageError: Line magic function `%%time` not found.


In [25]:
cluster_annotation_path = abc_cache.get_metadata_path(
    directory='WMB-taxonomy',
    file_name='cluster_annotation_term'
)

cluster_membership_path = abc_cache.get_metadata_path(
    directory='WMB-taxonomy',
    file_name='cluster_to_cluster_annotation_membership'
)

In [26]:
%%time

precompute_config = {
    'output_path': str(precompute_dir / 'precomputed_stats.h5'),
    'hierarchy': ['CCN20230722_CLAS',
                  'CCN20230722_SUBC',
                  'CCN20230722_SUPT',
                  'CCN20230722_CLUS'],
    'h5ad_path_list': h5ad_path_list,
    'cell_metadata_path': str(training_set_path),
    'cluster_annotation_path': str(cluster_annotation_path),
    'cluster_membership_path': str(cluster_membership_path),
    'n_processors': 4,
    'split_by_dataset': True,
    'do_pruning': True,
    'tmp_dir': str(scratch_dir),
    'clobber': True
}

precomputation_runner = PrecomputationABCRunner(
    args=[],
    input_data=precompute_config 
)

precomputation_runner.run()

CREATED TAXONOMY
writing /projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/precompute/precomputed_stats.WMB-10Xv2.h5 from dataset WMB-10Xv2
finally process 1566437 tot 3.18e+02 reading 5.27e+01 writing 2.88e+00
finally process 1566435 tot 3.44e+02 reading 5.78e+01 writing 2.50e+00
finally process 1566436 tot 3.50e+02 reading 6.52e+01 writing 2.55e+00
finally process 1566434 tot 3.56e+02 reading 5.90e+01 writing 2.47e+00
completed WMB-10Xv2 after 4.82e+02 seconds
writing /projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/precompute/precomputed_stats.WMB-10Xv3.h5 from dataset WMB-10Xv3
finally process 1567231 tot 1.52e+02 reading 2.72e+01 writing 2.55e+00
finally process 1567230 tot 1.97e+02 reading 3.88e+01 writing 2.44e+00
finally process 1567229 tot 2.10e+02 reading 4.04e+01 writing 2.60e+00
finally process 1567228 tot 2.11e+02 reading 4.54e+01 writing 2.52e+00
completed WMB-10Xv3 after 7.74e+02 seconds
merging
wrote /projectnb/selneuro/Shankar/code/Pyth

In [27]:
for file_path in precompute_dir.iterdir():
    print(file_path)

/projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/precompute/precomputed_stats.combined.h5
/projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/precompute/precomputed_stats.WMB-10Xv3.h5
/projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/precompute/precomputed_stats.WMB-10Xv2.h5


In [28]:
%%time

from cell_type_mapper.cli.reference_markers import (
    ReferenceMarkerRunner
)

precomputed_path_list = [
    str(p)
    for p in precompute_dir.iterdir()
    if not p.name.endswith('combined.h5')
]

reference_config = {
    'precomputed_path_list': precomputed_path_list,
    'output_dir': str(reference_dir),
    'tmp_dir': str(scratch_dir),
    'max_gb': 10,
    'n_processors': 4,
    'clobber': True
}

reference_runner = ReferenceMarkerRunner(
    args=[],
    input_data=reference_config
)

reference_runner.run()

writing /projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/reference/reference_markers.WMB-10Xv3.h5
Starting precomputed_stats.WMB-10Xv3.h5
111304 of 890445 taxon pairs in 1.10e+01 min; predict 7.71e+01 min of 8.81e+01 min left
222608 of 890445 taxon pairs in 1.37e+01 min; predict 4.12e+01 min of 5.49e+01 min left
333912 of 890445 taxon pairs in 1.41e+01 min; predict 2.35e+01 min of 3.76e+01 min left
445216 of 890445 taxon pairs in 1.49e+01 min; predict 1.49e+01 min of 2.98e+01 min left
556520 of 890445 taxon pairs in 2.05e+01 min; predict 1.23e+01 min of 3.28e+01 min left
667824 of 890445 taxon pairs in 2.05e+01 min; predict 6.83e+00 min of 2.73e+01 min left
779128 of 890445 taxon pairs in 2.35e+01 min; predict 3.36e+00 min of 2.69e+01 min left
890432 of 890445 taxon pairs in 2.42e+01 min; predict 3.54e-04 min of 2.42e+01 min left
1001736 of 890445 taxon pairs in 2.56e+01 min; predict -2.84e+00 min of 2.28e+01 min left
Initial marker discovery took 1.55e+03 seconds
join

In [29]:
for file_path in reference_dir.iterdir():
    print(file_path)

/projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/reference/reference_markers.WMB-10Xv2.h5
/projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/reference/reference_markers.WMB-10Xv3.h5


In [30]:
query_marker_path = query_dir / 'query_markers.json'


In [31]:
%%time

from cell_type_mapper.cli.query_markers import (
    QueryMarkerRunner
)

reference_marker_path_list = [
    str(p)
    for p in reference_dir.iterdir()
]

query_marker_config = {
    'output_path': str(query_marker_path),
    'reference_marker_path_list': reference_marker_path_list,
    'n_processors': 4,
    'tmp_dir': str(scratch_dir)
}

query_runner = QueryMarkerRunner(
    args=[],
    input_data=query_marker_config
)

query_runner.run()

found markers for 41 parents in 4.34e-01 minutes; predict 3.91e+00 of 4.35e+00 remaining
found markers for 82 parents in 5.74e-01 minutes; predict 2.30e+00 of 2.88e+00 remaining
found markers for 123 parents in 6.44e-01 minutes; predict 1.51e+00 of 2.15e+00 remaining
found markers for 165 parents in 7.40e-01 minutes; predict 1.10e+00 of 1.84e+00 remaining
found markers for 206 parents in 8.27e-01 minutes; predict 8.23e-01 of 1.65e+00 remaining
found markers for 247 parents in 9.03e-01 minutes; predict 6.00e-01 of 1.50e+00 remaining
found markers for 288 parents in 9.87e-01 minutes; predict 4.21e-01 of 1.41e+00 remaining
found markers for 330 parents in 1.04e+00 minutes; predict 2.55e-01 of 1.30e+00 remaining
found markers for 373 parents in 1.08e+00 minutes; predict 1.10e-01 of 1.19e+00 remaining
found markers for 24 parents in 5.19e-02 minutes; predict 4.72e-01 of 5.24e-01 remaining
found markers for 49 parents in 6.95e-02 minutes; predict 2.74e-01 of 3.43e-01 remaining
found markers 

In [32]:
from cell_type_mapper.cli.from_specified_markers import (
    FromSpecifiedMarkersRunner
)

In [33]:
%%time

json_output_path = '/projectnb/selneuro/Results/Patch_seq/map_my_cells/manual/json_mapping_output_branch_removed.json'
csv_output_path = '/projectnb/selneuro/Results/Patch_seq/map_my_cells/manual/csv_mapping_output_branch_removed.csv'

precompute_combined_path = precompute_dir / 'precomputed_stats.combined.h5'

mapping_config = {
    'query_path': '/projectnb/selneuro/Results/Patch_seq/map_my_cells/112_geneCounts_exon_plus_intron.h5ad',
    'extended_result_path': str(json_output_path),
    'csv_result_path': str(csv_output_path),
    'tmp_dir': str(scratch_dir),
    'max_gb': 10,
    'cloud_safe': False,
    'verbose_stdout': False,
    'type_assignment': {
        'normalization': 'raw',
        'n_processors': 4,
        'chunk_size': 10000,
        'bootstrap_iteration': 100,
        'bootstrap_factor': 0.5,
        'rng_seed': 233211
    },
    'precomputed_stats': {
        'path': str(precompute_combined_path)
    },
    'query_markers': {
        'serialized_lookup': str(query_marker_path)
    }
}

mapping_runner = FromSpecifiedMarkersRunner(
    args=[],
    input_data=mapping_config
)

mapping_runner.run()

=== Running Hierarchical Mapping 1.5.5 



/projectnb/selneuro/Shankar/code/Python/map_my_cells/cell_type_mapper/src/cell_type_mapper/taxonomy/utils.py:253: UserWarning: This taxonomy has no mapping from leaf_node -> rows in the cell by gene matrix
  warnings.warn("This taxonomy has no mapping from leaf_node -> rows "
/projectnb/selneuro/Shankar/code/Python/map_my_cells/cell_type_mapper/src/cell_type_mapper/utils/gene_utils.py:126: DuplicateGeneIDWarning: The following gene identifiers occurred more than once in your data. They will be ignored: ['ENSMUSG00000112376', 'ENSMUSG00000115230', 'ENSMUSG00000127913']
  warnings.warn(
/projectnb/selneuro/Shankar/code/Python/map_my_cells/cell_type_mapper/src/cell_type_mapper/cli/cli_log.py:73: UserWarning: 'CCN20230722_SUPT/CS20230722_SUPT_1165' has no valid markers in marker_lookup

  warnings.warn(msg)
/projectnb/selneuro/Shankar/code/Python/map_my_cells/cell_type_mapper/src/cell_type_mapper/cli/cli_log.py:73: UserWarning: parent node 'CCN20230722_SUPT/CS20230722_SUPT_1165' had too fe

MAPPING FROM SPECIFIED MARKERS RAN SUCCESSFULLY
CLEANING UP
CPU times: user 17.2 s, sys: 8.84 s, total: 26 s
Wall time: 31.5 s


In [20]:
!wget https://allen-brain-cell-atlas.s3-us-west-2.amazonaws.com/mapmycells/WMB-10X/20240831/mouse_markers_230821.json -O /projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/mouse_markers_230821.json
!wget https://allen-brain-cell-atlas.s3-us-west-2.amazonaws.com/mapmycells/WMB-10X/20240831/precomputed_stats_ABC_revision_230821.h5 -O /projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/precomputed_stats_ABC_revision_230821.h5

--2025-08-14 15:11:03--  https://allen-brain-cell-atlas.s3-us-west-2.amazonaws.com/mapmycells/WMB-10X/20240831/mouse_markers_230821.json
Resolving allen-brain-cell-atlas.s3-us-west-2.amazonaws.com (allen-brain-cell-atlas.s3-us-west-2.amazonaws.com)... 52.218.217.97, 52.218.182.9, 52.92.226.202, ...
Connecting to allen-brain-cell-atlas.s3-us-west-2.amazonaws.com (allen-brain-cell-atlas.s3-us-west-2.amazonaws.com)|52.218.217.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13510292 (13M) [application/json]
Saving to: ‘/projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/mouse_markers_230821.json’

/projectnb/selneuro 100%[===================>]  12.88M  10.7MB/s    in 1.2s    

2025-08-14 15:11:05 (10.7 MB/s) - ‘/projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/mouse_markers_230821.json’ saved [13510292/13510292]

--2025-08-14 15:11:06--  https://allen-brain-cell-atlas.s3-us-west-2.amazonaws.com/mapmycells/WMB-10X/20240831/precomput

In [21]:
precompute_path = '/projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/precomputed_stats_ABC_revision_230821.h5'
precompute_path

'/projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/precomputed_stats_ABC_revision_230821.h5'

In [38]:
import h5py
h5 = h5py.File(precompute_path, 'r')
print(h5)

<HDF5 file "precomputed_stats_ABC_revision_230821.h5" (mode r)>


In [39]:
print("Top-level keys:", list(h5.keys()))


Top-level keys: ['cluster_to_row', 'col_names', 'metadata', 'n_cells', 'sum', 'taxonomy_tree']


In [40]:
for key in h5.keys():
    item = h5[key]
    print(f"{key}: {'Group' if isinstance(item, h5py.Group) else 'Dataset'}")

cluster_to_row: Dataset
col_names: Dataset
metadata: Dataset
n_cells: Dataset
sum: Dataset
taxonomy_tree: Dataset


In [63]:
import h5py
import json

marker_json_path = '/projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/mouse_markers_230821.json'

with open(marker_json_path) as f:
    markers = json.load(f)

print("Top-level keys in marker JSON:")
print(list(markers.keys()))



Top-level keys in marker JSON:
['None', 'CCN20230722_CLAS/CS20230722_CLAS_01', 'CCN20230722_CLAS/CS20230722_CLAS_02', 'CCN20230722_CLAS/CS20230722_CLAS_03', 'CCN20230722_CLAS/CS20230722_CLAS_04', 'CCN20230722_CLAS/CS20230722_CLAS_05', 'CCN20230722_CLAS/CS20230722_CLAS_06', 'CCN20230722_CLAS/CS20230722_CLAS_07', 'CCN20230722_CLAS/CS20230722_CLAS_08', 'CCN20230722_CLAS/CS20230722_CLAS_09', 'CCN20230722_CLAS/CS20230722_CLAS_11', 'CCN20230722_CLAS/CS20230722_CLAS_10', 'CCN20230722_CLAS/CS20230722_CLAS_12', 'CCN20230722_CLAS/CS20230722_CLAS_13', 'CCN20230722_CLAS/CS20230722_CLAS_14', 'CCN20230722_CLAS/CS20230722_CLAS_16', 'CCN20230722_CLAS/CS20230722_CLAS_17', 'CCN20230722_CLAS/CS20230722_CLAS_18', 'CCN20230722_CLAS/CS20230722_CLAS_19', 'CCN20230722_CLAS/CS20230722_CLAS_20', 'CCN20230722_CLAS/CS20230722_CLAS_23', 'CCN20230722_CLAS/CS20230722_CLAS_24', 'CCN20230722_CLAS/CS20230722_CLAS_26', 'CCN20230722_CLAS/CS20230722_CLAS_27', 'CCN20230722_CLAS/CS20230722_CLAS_28', 'CCN20230722_CLAS/CS2023

In [46]:
marker_json_path = '/projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/mouse_markers_230821.json'

with open(marker_json_path) as f:
    markers = json.load(f)

target_class_path = 'CCN20230722_CLAS/CS20230722_CLAS_01'

# Get all class-level paths (those starting with 'CCN20230722_CLAS/')
all_class_paths = [k for k in markers.keys() if k.startswith('CCN20230722_CLAS/')]

# Prepare nodes_to_drop as list of (level, node) tuples. Use taxonomy level name as 'class'.
nodes_to_drop = [
    ('class', cls_path.split('/')[-1]) for cls_path in all_class_paths
    if cls_path != target_class_path
]

print(f"Number of class nodes to drop: {len(nodes_to_drop)}")
print("Example nodes to drop:", nodes_to_drop[:5])

Number of class nodes to drop: 28
Example nodes to drop: [('class', 'CS20230722_CLAS_02'), ('class', 'CS20230722_CLAS_03'), ('class', 'CS20230722_CLAS_04'), ('class', 'CS20230722_CLAS_05'), ('class', 'CS20230722_CLAS_06')]


Loaded counts with shape: (56951, 112)


In [60]:
import anndata
import scipy.sparse
import pandas as pd

csv_path = '/projectnb/selneuro/Results/Patch_seq/bcl_convert/geneCounts_exon_plus_intron_ensembl.csv'

# If your CSV has genes as row indices and cells as columns (or vice versa),
# adjust accordingly. Assuming genes are rows and columns are cell IDs:
counts_df = pd.read_csv(csv_path, index_col=0)

print("Loaded counts with shape:", counts_df.shape)
#print(counts_df.head())
# Transpose so cells are rows and genes are columns
counts_df_T = counts_df.T

print(f"Transposed shape (cells x genes): {counts_df_T.shape}")

# Convert to sparse matrix (to save memory)
counts_sparse = scipy.sparse.csr_matrix(counts_df_T.values)

# Create AnnData object
adata = anndata.AnnData(X=counts_sparse)

# Set obs (cells) and var (genes) names
adata.obs_names = counts_df_T.index.tolist()  # cells
adata.var_names = counts_df_T.columns.tolist()  # genes

adata.write('/projectnb/selneuro/Results/Patch_seq/map_my_cells/112_geneCounts_exon_plus_intron.h5ad')
print("AnnData saved, ready for mapping with cell_type_mapper.")

Loaded counts with shape: (46891, 112)
Transposed shape (cells x genes): (112, 46891)
AnnData saved, ready for mapping with cell_type_mapper.


AnnData saved, ready for mapping with cell_type_mapper.


In [51]:
import json

marker_json_path = '/projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/mouse_markers_230821.json'

with open(marker_json_path) as f:
    markers = json.load(f)

target_class_path = 'CCN20230722_CLAS/CS20230722_CLAS_01'

# Extract all class paths
all_class_paths = [k for k in markers.keys() if k.startswith('CCN20230722_CLAS/')]

# Prepare nodes to drop (all except your target class)
nodes_to_drop = [
    ('class', cls_path.split('/')[-1]) for cls_path in all_class_paths
    if cls_path != target_class_path
]

print(f"Dropping {len(nodes_to_drop)} classes, keeping only CS20230722_CLAS_01.")
print("Example nodes to drop:", nodes_to_drop[:5])

Dropping 28 classes, keeping only CS20230722_CLAS_01.
Example nodes to drop: [('class', 'CS20230722_CLAS_02'), ('class', 'CS20230722_CLAS_03'), ('class', 'CS20230722_CLAS_04'), ('class', 'CS20230722_CLAS_05'), ('class', 'CS20230722_CLAS_06')]


In [59]:
import pandas as pd
from mygene import MyGeneInfo

# Load your CSV
df = pd.read_csv("/projectnb/selneuro/Results/Patch_seq/bcl_convert/combined_oct_may_jul/analyze/count_matrices_umionly/geneCounts_exon_plus_intron.csv", index_col=0)

mg = MyGeneInfo()

# Get your gene symbols (assuming index is gene symbol)
gene_symbols = df.index.tolist()

# Query mygene for mouse gene symbols to Ensembl IDs
query_results = mg.querymany(gene_symbols, scopes='symbol', fields='ensembl.gene', species='mouse')

# Create a mapping dictionary from the results
mapping = {}
for entry in query_results:
    if 'ensembl' in entry:
        # ensembl might be a list of dicts or a single dict
        if isinstance(entry['ensembl'], list):
            ensembl_id = entry['ensembl'][0]['gene']
        else:
            ensembl_id = entry['ensembl']['gene']
        mapping[entry['query']] = ensembl_id
    else:
        mapping[entry['query']] = None

# Map the index of your dataframe to ensembl IDs
df['ensembl_id'] = df.index.to_series().map(mapping)

# Drop rows with no mapping if desired
df = df[~df['ensembl_id'].isnull()]

# Set index to Ensembl IDs
df.index = df['ensembl_id']
df = df.drop(columns=['ensembl_id'])

# Save to a new CSV
df.to_csv("/projectnb/selneuro/Results/Patch_seq/bcl_convert/geneCounts_exon_plus_intron_ensembl.csv")

INFO:biothings.client:querying 1-1000 ...
INFO:biothings.client:querying 1001-2000 ...
INFO:biothings.client:querying 2001-3000 ...
INFO:biothings.client:querying 3001-4000 ...
INFO:biothings.client:querying 4001-5000 ...
INFO:biothings.client:querying 5001-6000 ...
INFO:biothings.client:querying 6001-7000 ...
INFO:biothings.client:querying 7001-8000 ...
INFO:biothings.client:querying 8001-9000 ...
INFO:biothings.client:querying 9001-10000 ...
INFO:biothings.client:querying 10001-11000 ...
INFO:biothings.client:querying 11001-12000 ...
INFO:biothings.client:querying 12001-13000 ...
INFO:biothings.client:querying 13001-14000 ...
INFO:biothings.client:querying 14001-15000 ...
INFO:biothings.client:querying 15001-16000 ...
INFO:biothings.client:querying 16001-17000 ...
INFO:biothings.client:querying 17001-18000 ...
INFO:biothings.client:querying 18001-19000 ...
INFO:biothings.client:querying 19001-20000 ...
INFO:biothings.client:querying 20001-21000 ...
INFO:biothings.client:querying 2100

In [61]:
from cell_type_mapper.cli.from_specified_markers import FromSpecifiedMarkersRunner
import pathlib

#scratch_dir = pathlib.Path('/path/to/scratch/')

mapping_config = {
    'query_path': '/projectnb/selneuro/Results/Patch_seq/map_my_cells/112_geneCounts_exon_plus_intron.h5ad',  # Your AnnData query data
    'extended_result_path': str('/projectnb/selneuro/Results/Patch_seq/map_my_cells/manual/json_mapping_output_class01.json'),
    'csv_result_path': str('/projectnb/selneuro/Results/Patch_seq/map_my_cells/manual/csv_mapping_output_class01.csv'),
    'tmp_dir': str(scratch_dir),
    'max_gb': 10,
    'cloud_safe': False,
    'verbose_stdout': True,
    'type_assignment': {
        'normalization': 'raw',  # or 'log' depending on your data prep
        'n_processors': 4,
        'chunk_size': 10000,
        'bootstrap_iteration': 100,
        'bootstrap_factor': 0.5,
        'rng_seed': 12345
    },
    'precomputed_stats': {
        'path': '/projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/precomputed_stats_ABC_revision_230821.h5',
    },
    'query_markers': {
        'serialized_lookup': '/projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/mouse_markers_230821.json',
    },
    'nodes_to_drop': nodes_to_drop,
    'drop_level': 'class',  # same level as in nodes_to_drop
}

mapping_runner = FromSpecifiedMarkersRunner(args=[], input_data=mapping_config)
mapping_runner.run()

=== Running Hierarchical Mapping 1.5.5 with config ===
{
  "verbose_csv": false,
  "log_level": "ERROR",
  "extended_result_path": "/projectnb/selneuro/Results/Patch_seq/map_my_cells/manual/json_mapping_output_class01.json",
  "csv_result_path": "/projectnb/selneuro/Results/Patch_seq/map_my_cells/manual/csv_mapping_output_class01.csv",
  "verbose_stdout": true,
  "query_gene_id_col": null,
  "obsm_clobber": false,
  "drop_level": "class",
  "max_gb": 10.0,
  "summary_metadata_path": null,
  "cloud_safe": false,
  "query_markers": {
    "serialized_lookup": "/projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch/mouse_markers_230821.json",
    "log_level": "ERROR",
    "collapse_markers": false
  },
  "log_path": null,
  "tmp_dir": "/projectnb/selneuro/Shankar/code/Python/map_my_cells/data/scratch",
  "flatten": false,
  "nodes_to_drop": [
    [
      "class",
      "CS20230722_CLAS_02"
    ],
    [
      "class",
      "CS20230722_CLAS_03"
    ],
    [
      "class",
      "

/projectnb/selneuro/Shankar/code/Python/map_my_cells/cell_type_mapper/src/cell_type_mapper/taxonomy/utils.py:253: UserWarning: This taxonomy has no mapping from leaf_node -> rows in the cell by gene matrix
  warnings.warn("This taxonomy has no mapping from leaf_node -> rows "


FILE TRACKER: copied ../112_geneCounts_exon_plus_intron.h5ad to ../112_geneCounts_exon_plus_intron_ilc2dzdy.h5ad
BENCHMARK: spent 3.3463e-02 seconds validating config and copying data
using ../precomputed_stats_ABC_revision_230821.h5fzu1lcko.h5 for precomputed_stats
reading taxonomy_tree from ../precomputed_stats_ABC_revision_230821.h5fzu1lcko.h5


/projectnb/selneuro/Shankar/code/Python/map_my_cells/cell_type_mapper/src/cell_type_mapper/utils/gene_utils.py:126: DuplicateGeneIDWarning: The following gene identifiers occurred more than once in your data. They will be ignored: ['ENSMUSG00000112376', 'ENSMUSG00000115230', 'ENSMUSG00000127913']
  warnings.warn(
/projectnb/selneuro/Shankar/code/Python/map_my_cells/cell_type_mapper/src/cell_type_mapper/cli/cli_log.py:73: UserWarning: 'CCN20230722_SUPT/CS20230722_SUPT_1185' not listed in marker lookup

  warnings.warn(msg)
/projectnb/selneuro/Shankar/code/Python/map_my_cells/cell_type_mapper/src/cell_type_mapper/cli/cli_log.py:73: UserWarning: parent node 'CCN20230722_SUPT/CS20230722_SUPT_1185' had too few markers in query set; augmenting with markers from ['CCN20230722_SUBC/CS20230722_SUBC_328']
  warnings.warn(msg)
/projectnb/selneuro/Shankar/code/Python/map_my_cells/cell_type_mapper/src/cell_type_mapper/cli/cli_log.py:73: UserWarning: 'CCN20230722_SUPT/CS20230722_SUPT_0892' not liste

BENCHMARK: spent 2.0657e+00 seconds creating query marker cache
Running CPU implementation of type assignment.


/projectnb/selneuro/Shankar/code/Python/map_my_cells/cell_type_mapper/src/cell_type_mapper/diff_exp/score_utils.py:303: UserWarning: precomputed stats file does not have 'ge1' data
  warnings.warn("precomputed stats file does not have 'ge1' data")


BENCHMARK: spent 5.7746e+00 seconds assigning cell types
Writing marker genes to output file
FILE TRACKER: cleaning up ../file_tracker_kkxw_kta
MAPPING FROM SPECIFIED MARKERS RAN SUCCESSFULLY
CLEANING UP
